In [ ]:
import sys
from pathlib import Path

In [ ]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [ ]:
sys.path.append(str(Path("../ssl").resolve()))

In [ ]:
from src.models.lenet5.lenet5 import Lenet5
from src.models.lenet5.lenet5_config import Lenet5Config
from src.trainers.basic.categorical_ce import CategoricalCETrainer
from src.trainers.basic.categorical_ce_config import CategoricalCETrainerConfig
from src.data_loaders.basic.categorical_ce_config import CategoricalCEDataLoaderConfig
from src.data_loaders.basic.categorical_ce import CategoricalCEDataLoader

## Set up Experiment

In [ ]:
class TrainerConfig(CategoricalCETrainerConfig):
    num_epochs = 5
    
    # lr_schedule will override fixed optimizer learning rate
    lr_schedule = {
        "name": "exponential",
        "params": {
            "initial_learning_rate": 1e-4,
            "decay_steps": 10000,
            "decay_rate": 0.90
        }
    }
    optimizer = {
        "name": "adam",
        "learning_rate": 1e-2,
        "params": {
        }
    }

train_config = TrainerConfig()

In [ ]:
class ModelConfig(Lenet5Config):
    input_shape = (32, 32, 3)
    output_shape = 10

model_config = ModelConfig()

In [ ]:
class DataLoaderConfig(CategoricalCEDataLoaderConfig):
    batch_size = 64
    num_classes = 10

data_loader_config = DataLoaderConfig()

## Get Datasets

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train,
    stratify = y_train, test_size = 0.20, random_state = 42)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))

In [ ]:
# create train dataset
train_data = CategoricalCEDataLoader(train_data, data_loader_config)(training = True)

In [ ]:
# create test dataset
val_data = CategoricalCEDataLoader(val_data, data_loader_config)(training = False)

# Train Model

In [ ]:
model = Lenet5(model_config)()

In [ ]:
trainer = CategoricalCETrainer(
    model, train_data, train_config,
    val_dataset = val_data
)

In [ ]:
trainer.train()